In [ ]:
"""
I guess I have to start with describing the key elements in this game
    Classes: Card, Deck, Player
    Actions: Deal, Bet, Fold
    Add-ons: Score, Pool

This was created to learn Python - the interface shows cards of all players

"""

In [ ]:
# Importing some basic things

import random
from IPython.display import clear_output
from termcolor import colored

In [ ]:
# Setting the variables

suits = {'Hearts':colored('♥','red',attrs=['dark','bold']), 'Spades':'♠', 
         'Diamonds':colored('♦','red',attrs=['dark','bold']), 'Clubs':'♣'}

ranks = {'Two':[2,'2'], 'Three':[3,'3'], 'Four':[4,'4'], 'Five':[5,'5'],
         'Six':[6,'6'], 'Seven':[7,'7'], 'Eight':[8,'8'],'Nine':[9,'9'],
         'Ten':[10,'10'], 'Jack':[11,'J'], 'Queen':[12,'Q'], 'King':[13,'K'], 'Ace':[14,'A']}

In [ ]:
# Creating the classes

class Card:
    def __init__ (self,suit,rank):
        self.suit = suit
        self.rank = rank
        self.value = ranks[rank][0]
    def __str__ (self):
        return ranks[self.rank][1] + suits[self.suit]

class Deck:
    def __init__ (self):
        self.pack = []
        for suit in suits:
            for rank in ranks:
                self.pack.append(Card(suit,rank))
        random.shuffle(self.pack)
    def deal (self):
        return self.pack.pop()
    
class Player:
    def __init__ (self,name,stack):
        self.name = name
        self.stack = stack
        self.cards = []
        self.hand = ''
        self.bets = 0
    def __str__ (self):
        return self.hand
    def get_card (self,card):
        self.hand += str(card) + ' '
        self.cards.append(card)
    def bet(self,size):
        global table
        if size >= self.stack:
            temp_size = self.stack
            print(f'{self.name} goes ALL-IN with ${int(self.stack)}!')
            self.bets += self.stack
            self.stack = 0
            table.stack += temp_size
            return temp_size
        else:
            print(f'{self.name} bets ${int(size)}')
            self.bets += size
            self.stack -= size
            table.stack += size
            return size
    def fold(self):
        print(f'{self.name} folds')
        self.cards = []
    def check(self):
        print(f'{self.name} checks')
    def win(self,size):
        global table
        print(f'{self.name} wins ${int(size)} - Congratulations!')
        self.stack += size
        table.stack -= size
    def refresh(self):
        self.cards = []
        self.hand = ''
        self.bets = 0

In [ ]:
# Setting up the table

def allocate(num,stack):
    global players
    
    for x in range(num):
        players.append(Player(input("Enter player's name: "),stack))
    
    # Select dealer
    dealer = 0
    local_deck = Deck()
    local_players = [p for p in players]
    
    while len(local_players) > 1:
        local_kickers = []
        kill_list = []
        
        # deal cards
        for p in local_players:
            card = local_deck.deal()
            p.get_card(card)
            local_kickers.append(card.value)
            print(f'{p.name} has {card}')
        
        # check if one dealer
        for p in local_players:
            if p.cards[0].value == max(local_kickers) and local_kickers.count(p.cards[0].value) == 1:
                dealer = players.index(p)
            if p.cards[0].value != max(local_kickers):
                kill_list.append(p)
        
        # remove losers
        for p in kill_list:
            local_players.remove(p)
                
    # Rotate players
    for x in range(dealer+1):
        players.append(players.pop(0))
        
    # Return small blind
    blind = stack / 100
    print(f'{players[-1].name} will be a dealer')
    return blind
        

In [ ]:
# Logic of the game

def game(num):
    
    global players
    players = []
    global table
    table = Player('table',0)
    global blind
    blind = allocate(num,1000)

    while len(players) != 1:

        global deck
        deck = Deck()
        table.refresh()
        for p in players:
            p.refresh()

        clear_output()
        
        print('\n>> Pre-flop')
        circle(0)
        
        print('\n>> Flop')
        circle(3)
        
        print('\n>> Turn')
        circle(1)
        
        print('\n>> River')
        circle(1)

        distribute_pool(players)
        
        # Ask if players want to continue
        decision = 'a'
        answers = ['y','n']
        while decision not in answers: 
            decision = input('Continue? y or n: ')
            
        # Break if they don't
        if decision == 'n':
            print(f'Game interrupted, final standings are:')
            for p in players:
                print(f'{p.name} has ${str(p.stack)}')
            break
        
        players.append(players.pop(0))
        players = [p for p in players if p.stack>0]
        
    else:
        winner = [p for p in players if p.stack>0]
        print(f'{winner[0].name} won the game like a boss!\nGame over..')
            


In [ ]:
# One round

def circle(cards_dealt):
    
    if cards_dealt == 0: # Pre-flop

        # Blinds & cards
        for x in range(2):
            players[x].bet(blind*(x+1))
            for p in players:
                p.get_card(deck.deal())

        show_status()

        # Remaining
        keep_running_bets(players[2:]+players[:2])
        print(f'Bets accepted, the pool is ${int(table.stack)}')

    else: # Flop / River / Turn

        # To make sure nothing is executed for 1 player only
        active_players = [x for x in players if x.cards!=[]]
        if len(active_players) > 1:
             
            for x in range(cards_dealt):
                table.get_card(deck.deal())

            show_status()

            keep_running_bets(active_players)
            print(f'Bets accepted, the pool is ${int(table.stack)}')

def show_status():
    print(f'Table cards: {str(table)}')
    print(f'Table pool: ${int(table.stack)}')
    for p in [a for a in players if a.cards !=[]]:
        print(f'{p.name} has {str(p)} with total bets of ${int(p.bets)} and stack worth ${int(p.stack)}')
    
    print('\n')

In [ ]:
# Underlying perpetual betting loop

def keep_running_bets(list_of_players):

    # Set local pool
    local_pool = {x:0 for x in list_of_players if x.cards!=[] and x.stack>0}
    who_raised = []
        
    # Pre-flop blinds
    if len(table.cards) == 0:
        for player,bet in local_pool.items():
            local_pool[player] = player.bets
    
    # Take bets
    while True:
        active_players = [p for p in list_of_players if p.cards!=[] and p.stack>0]
        for player in active_players:

            max_bet = max(local_pool.values())
            
            # Check if this was the guy who initiated raises
            if len(who_raised)>0:    
                if local_pool[player] == max_bet and who_raised[-1] == player:
                    break
            
            # Check if only one player not all in
            if len(active_players) == 1:
                break
            
            allowed_bets = ['c','r','f']
            local_bet = 'a'
            
            while local_bet not in allowed_bets:
                local_bet = input(f'{player.name}, your move: ')

            # Check or Call
            if local_bet == 'c' and  local_pool[player] == max_bet:
                player.check()
            elif local_bet == 'c':
                local_pool[player] += player.bet(max_bet - local_pool[player])
            
            # Raise
            if local_bet == 'r':
                size = 0
                min_raise = int(max(blind*2,max_bet*2))
                while size < min_raise:
                    try:
                        size = int(input(f'Please make a bet (minimum ${min_raise}): '))
                    except:
                        if min_raise > player.stack:
                            break
                local_pool[player] += player.bet(min(size,player.stack))
                who_raised.append(player)
            
            # Fold
            if local_bet == 'f':
                player.fold()
            
        # Break if all equal
        local_bets = [x.bets for x in active_players if x.cards!=[] and x.stack>0]
        if len(set(local_bets))<2:
            break


In [ ]:
def distribute_pool(list_of_players):
    
    those_with_cards = [p for p in list_of_players if p.cards != []]
    
    if len(those_with_cards) == 1:
        those_with_cards[0].win(table.stack)
        return 'Everybody folded'
    
    # final dic for last statement = {player: win amount}}
    final_winners = {p:0 for p in list_of_players}
    
    # Create pools for each set of bets
    set_of_bets = sorted(set([p.bets for p in list_of_players]))
    pools=[]
    distributed = 0
    num_of_pools = len(set_of_bets)
    for bet in range(num_of_pools):
        current_bet = set_of_bets.pop(0) - distributed
        pools.append(current_bet)
        distributed += current_bet
    
    # Distribute each pool
    for pool in pools:
        
        # who contends
        contenders = [p for p in list_of_players if p.bets >= pool]
        
        # size of pool
        pool_size = pool * len(contenders)
        
        # who wins
        max_score = max([get_score(p) for p in contenders if p.cards != []])
        winners = [p for p in contenders if get_score(p) == max_score and p.cards != []]
        
        # how much
        win_size = pool_size / len(winners)
        for p in winners:
            final_winners[p] += win_size
            
        # resize
        for p in contenders:
            p.bets -= pool

    
    # final declaration
    combos = ['kicker','pair','two pairs','three of a kind','street','flush','full house','four of a kind','straight flush']
    for player,pool in final_winners.items():
        if final_winners[player] > 0:
            print(f'{player.name} has {combos[int(get_score(player))-1]}')
            player.win(pool)

    

In [ ]:
# Calculating a score

def get_score(player):
    comb_cards = player.cards + table.cards
    combinations = []
    
    # street
    values = sorted(set([c.value for c in comb_cards]))
    if 14 in values: # adding the Ace to the other side
        values.append(1)
    local_streets = []
    if len(values)>4:
        for x in range(len(values)):
            if values[0] == values[1]-1 == values[2]-2 == values[3]-3 == values[4]-4:
                local_streets.append(values[4])
            values.append(values.pop(0))
        if len(local_streets)>0:
            street_score = 5 + (max(local_streets) / 100)
            combinations.append(street_score)
    
    # flush
    flushes = sorted(set([c.value for c in comb_cards if [ca.suit for ca in comb_cards].count(c.suit) >=5])) 
    if len(flushes)>0:
        flush_score = 6
        for x in range(1,len(flushes)+1):
            flush_score += flushes.pop() / 10**(x*2)
        combinations.append(flush_score)
    
    # street flush
    local_street_flush = []
    
    if len(local_streets)>0 and len(flushes)>0:
        
        new_values = sorted(set([c.value for c in comb_cards if c.value in flushes]))
        
        if len(new_values)>4:
            for x in range(len(new_values)):
                if new_values[0] == new_values[1]-1 == new_values[2]-2 == new_values[3]-3 == new_values[4]-4:
                    local_street_flush.append(new_values[4])
                new_values.append(new_values.pop(0))

    
    if len(local_street_flush)>0:
        st_flush_score = 9 + (max(local_street_flush) / 100)
        combinations.append(st_flush_score)
    
    # pairs
    pairs = sorted(set([c.value for c in comb_cards if [ca.value for ca in comb_cards].count(c.value) == 2]))
    if len(pairs) == 1:
        kicker_pairs = sorted([x.value for x in comb_cards if x.value not in pairs])
        pair_score = 2 + (pairs[0] / 100) + (kicker_pairs.pop() / 10000) + (kicker_pairs.pop() / 1000000) + (kicker_pairs.pop() / 100000000)
        combinations.append(pair_score)
    
    # two pairs
    if len(pairs) >1:
        top_two_pairs = []
        for x in range(2):
            top_two_pairs.append(pairs.pop())
        two_pairs_kicker = max([x.value for x in comb_cards if x.value not in top_two_pairs])
        two_pair_score = 3 + (max(top_two_pairs) / 100) + (min(top_two_pairs) / 10000) + (two_pairs_kicker / 1000000)
        combinations.append(two_pair_score)
    
    # set
    sets = set([c.value for c in comb_cards if [ca.value for ca in comb_cards].count(c.value) == 3])
    if len(sets)>0:
        kickers_set = sorted([x.value for x in comb_cards if x.value != max(sets)])
        set_score = 4 + (max(sets) / 100) + (kickers_set.pop() / 10000) + (kickers_set.pop() / 1000000)
        combinations.append(set_score)
    
    # four of a kind
    kares = set([c.value for c in comb_cards if [ca.value for ca in comb_cards].count(c.value) == 4])
    if len(kares)>0:
        kare_kicker = max([x.value for x in comb_cards if x.value != max(kares)])
        kare_score = 8 + (kare_kicker / 100)
        combinations.append(kare_score)
    
    # full house
    local_full_house = []
    if len(pairs) > 0 and len(sets) > 0:
        full_house_score = 7 + (max(sets) / 100) + (max(pairs) / 10000)
        combinations.append(full_house_score)
        
    # kickers
    kickers = sorted(set([x.value for x in comb_cards]))
    kicker_score = 1
    for x in range(1,len(kickers)+1):
        kicker_score += kickers.pop() / 10**(x*2)
    combinations.append(kicker_score)
    
    
    # declare combination
    final_score = max(combinations)

    
    return final_score
        

In [ ]:
game(4) # indicate number of players 